In [1]:
import tensorflow as tf
import numpy as np
import time

print(tf.__version__)

2.0.0


In [47]:
import traceback
import contextlib


# 追踪不同类型错误
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception(
            'Expected {} to be raised but no error was raised!'.format(
                error_class))

`tf.function` 模块，集合 `AutoGraph` 机制，通过 `@tf.function` 修饰符将模型转换成图模式执行
> - 被其修饰的函数内，可使用的语句有一定的限制，且操作本身能够被构建为计算图
- 在函数内只使用 `TensorFlow` 的原生操作，不要使用过于复杂的 `Python` 语句，函数参数只包括 `TensorFlow` 张量或 `NumPy` 数组

# 使用示例

In [2]:
############### 简单模型 #####################
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu,   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu,
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output
    
    
    
############## 数据管道 #############################
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]
    
############## 创建模型 ###################################    
num_batches = 1000
batch_size = 50
learning_rate = 0.001
data_loader = MNISTLoader()
model = CNN()
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

In [6]:
############## 使用tf.function修饰，训练模型 ###################################
@tf.function
def train_one_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y,
                                                               y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        # 使用 TensorFLow 内置的 tf.print 函数，tf.function 不支持 Python 内置 print
        tf.print("loss: ", loss)
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))


start_time = time.time()
for batch_idx in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    train_one_step(X, y)
end_time = time.time()
print(end_time - start_time)

loss:  2.30859
loss:  2.2340374
loss:  2.11493564
loss:  1.93809342
loss:  1.83007169
loss:  1.69274735
loss:  1.59957325
loss:  1.21699929
loss:  1.15978587
loss:  0.932443619
loss:  0.872278512
loss:  0.679867625
loss:  0.516669095
loss:  0.758668065
loss:  0.751002252
loss:  0.664085925
loss:  0.652081609
loss:  0.821466625
loss:  0.773144126
loss:  0.57678324
loss:  0.268536806
loss:  0.53191483
loss:  0.456898868
loss:  0.415572137
loss:  0.239215359
loss:  0.53759706
loss:  0.439769208
loss:  0.266360849
loss:  0.601416
loss:  0.343753695
loss:  0.201910421
loss:  0.312974066
loss:  0.306883037
loss:  0.363924
loss:  0.200239
loss:  0.360672683
loss:  0.238687128
loss:  0.236905187
loss:  0.345142633
loss:  0.138407245
loss:  0.42357564
loss:  0.132109851
loss:  0.219590038
loss:  0.490732968
loss:  0.278162152
loss:  0.15120703
loss:  0.0375153571
loss:  0.20297794
loss:  0.338969618
loss:  0.150947526
loss:  0.17179352
loss:  0.223425925
loss:  0.196491361
loss:  0.285376579
lo

loss:  0.0983503833
loss:  0.0321525298
loss:  0.0576387
loss:  0.0706933
loss:  0.146571547
loss:  0.0688927844
loss:  0.277490675
loss:  0.0136702899
loss:  0.0127472868
loss:  0.0142394518
loss:  0.0936196446
loss:  0.0104037765
loss:  0.0436459146
loss:  0.0147388466
loss:  0.077041775
loss:  0.058235947
loss:  0.0146927489
loss:  0.0302243158
loss:  0.114086412
loss:  0.0888605416
loss:  0.0392818637
loss:  0.0522052273
loss:  0.0338935107
loss:  0.0436260477
loss:  0.139165312
loss:  0.0417692177
loss:  0.00542151788
loss:  0.00965692382
loss:  0.224222168
loss:  0.250543982
loss:  0.0408216082
loss:  0.0615965463
loss:  0.0886703879
loss:  0.0464459807
loss:  0.119751126
loss:  0.0518451594
loss:  0.0195443276
loss:  0.0350716673
loss:  0.105336994
loss:  0.0105956923
loss:  0.01623776
loss:  0.0366701782
loss:  0.0577814728
loss:  0.0528233238
loss:  0.0225539543
loss:  0.0375950933
loss:  0.0276213437
loss:  0.0357109942
loss:  0.0221369602
loss:  0.012641822
loss:  0.05100880

loss:  0.0100687006
loss:  0.0291222986
loss:  0.0746133626
loss:  0.0286083464
loss:  0.122451335
loss:  0.0526708215
loss:  0.115209013
loss:  0.0918994248
loss:  0.0442525
loss:  0.108096033
loss:  0.0254177097
loss:  0.0141457915
loss:  0.0393474549
loss:  0.00619741809
loss:  0.0717821196
loss:  0.0183010045
loss:  0.0928830355
loss:  0.100961208
loss:  0.148157611
loss:  0.0459507
loss:  0.0329650491
loss:  0.0559365116
loss:  0.0957076848
loss:  0.00941165071
loss:  0.00987342373
loss:  0.109686993
loss:  0.0426589437
loss:  0.150195509
loss:  0.0358754806
loss:  0.0585423522
loss:  0.0219188705
loss:  0.0996154696
loss:  0.045648586
loss:  0.0769954696
loss:  0.0130854975
loss:  0.018405892
loss:  0.0186882261
loss:  0.0101105096
loss:  0.0595490448
loss:  0.0631452352
loss:  0.0252163224
loss:  0.00694755418
loss:  0.0256930534
loss:  0.0124889351
loss:  0.0889802
loss:  0.024431251
loss:  0.0306418035
loss:  0.00617296621
loss:  0.17561537
loss:  0.108031541
loss:  0.01101746

> 上例中，被修饰函数参数 `X,y` 皆为 Numpy 数组；   
内部通过 TensorFlow 函数定义了 `loss` 变量；内部引用了外部对象 `model,optimizer`，皆为 TensorFlow 对象；相关操作皆为 TensorFlow 原生操作

# `tf.function` 内在机制

当被 `@tf.function` 修饰的函数第一次被调用的时候:
1. `tf.function` 不支持的语句会直接执行，**并不会转换成计算图中的节点**
2. 而每个 `tf.` 方法都只是定义了计算节点，而并没有进行任何实质的计算
3. `Python` 控制流语句转换成 TensorFlow 计算图中的对应节点（比如说 `while` 和 `for` 语句转换为 `tf.while` ， `if` 语句转换为 `tf.cond` 等等；
4. 基于上两部步，建立函数内代码的计算图表示
5. 运行一次该计算图
6. 将计算图缓存起来  

当再次调用函数时，直接运行计算图
- **注意：此时原始函数中`tf.function` 不支持的语句，已经不再存在**

> **提示**：使用被修饰函数的 `get_concrete_function` 方法，可以直接获得生成的计算图。该方法接受的参数与原始函数相同  
`graph=train_one_step.get_concrete_function(X,y)`

## 传入参数

In [21]:
@tf.function
def f(x):
    tf.print(x)
    print("The function is running in Python")    

In [12]:
#### 第一次调用函数
x = tf.constant(1, dtype=tf.int32)
f(x)

The function is running in Python
1


> 第一次执行：
1. 输入参数 `x` 为 TensorFlow 的 `tf.int32` 张量
- 尽管 `tf.print` 函数在前，但不会直接执行，而是定义节点
- `print` 函数会直接执行
- 图建立完成，然后执行，`tf.print` 函数

In [13]:
#### 第二次调用函数
x = tf.constant(10, dtype=tf.int32)
f(x)

10


> 第二次执行：
1. 输入参数 `x` 仍然为 TensorFlow 的 `tf.int32` 张量
- 直接执行图，即运行`tf.print` 函数，而`print`函数不包含在图中

In [14]:
#### 再次调用
b_ = np.array(2, dtype=np.int32) 
f(b_)

2


> 再次执行时：
1. 输入参数 `x` 变为 `np.int32` 格式
- 仍然直接执行图，即运行`tf.print` 函数，而`print`函数不包含在图中

In [16]:
#### 改变数据类型，再次调用函数
c = tf.constant(0.1, dtype=tf.float32)
f(c)
d = tf.constant(0.2, dtype=tf.float32)
f(d)

The function is running in Python
0.1
0.2


> 输入数据格式改变，再次执行函数时：
1. 因为**输入数据格式改变，重新创建计算图**，然后运行
- 重复调用，即重复执行图

In [22]:
#### 输入原生的 python 数据
f(d)
f(1)
f(2)
f(1)
f(0.1)
f(0.2)
f(0.1)

The function is running in Python
0.2
The function is running in Python
1
The function is running in Python
2
1
The function is running in Python
0.1
The function is running in Python
0.2
0.1


> 对于 Python 内置的整数和浮点数类型，只有当值完全一致的时候， `@tf.function` 才会复用之前建立的计算图，而并不会自动将 Python 内置的整数或浮点数等转换成张量

## 调用外部变量

In [23]:
#### 函数内使用了外部变量
a = tf.Variable(0.0)

@tf.function
def g():
    a.assign(a + 1.0)
    return a

print(g())
print(g())
print(g())

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(3.0, shape=(), dtype=float32)


> 在被 `@tf.function` 修饰的函数里调用 `tf.Variable` 、 `tf.keras.optimizers` 、 `tf.keras.Model` 等包含有变量的数据结构  
一旦被调用，这些结构将作为隐含的参数提供给函数。当这些结构内的值在函数内被修改时，在函数外也同样生效。

## Python 控制流

In [26]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x


a = tf.constant(2)
b = tf.constant(-2)
print(square_if_positive(a), square_if_positive(b))

tf.Tensor(4, shape=(), dtype=int32) tf.Tensor(0, shape=(), dtype=int32)


In [27]:
print(tf.autograph.to_code(square_if_positive.python_function))

def tf__square_if_positive(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('square_if_positive', 'square_if_positive_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as square_if_positive_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def if_true():
      x_1, = x,
      x_1 = x_1 * x_1
      return x_1

    def if_false():
      x = 0
      return x
    cond = x > 0
    x = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('x',), ())
    do_return = True
    retval_ = square_if_positive_scope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)



> - 原函数中的 Python 控制流 `if...else...` 被转换为了 `x = ag__.if_stmt(cond, if_true, if_false, get_state, set_state)` 这种计算图式的写法
- 即 `tf.function` 支持 python 控制流，因此可以直接使用 Python 控制流构建计算图，而不是手动使用 TensorFlow 的 API

## `tf.TensorArray`：TensorFlow 动态数组

- 在即时执行模式下，可以直接使用一个 Python 列表（List）存放数组。但需要基于计算图的特性（例如使用 `@tf.function` 加速模型运行或者使用 `SavedModel` 导出模型），就无法使用这种方式了
- TensorFlow 提供了 `tf.TensorArray` ，一种支持计算图特性的 TensorFlow 动态数组。



- 其声明的方式为：
    - `arr = tf.TensorArray(dtype, size, dynamic_size=False)` ：声明一个大小为 `size` ，类型为 `dtype` 的 T`ensorArray arr` 。如果将 `dynamic_size` 参数设置为 `True` ，则该数组会自动增长空间。
   
      
- 其读取和写入的方法为：

    - `write(index, value)` ：将 `value `写入数组的第 `index` 个位置；

    - `read(index)` ：读取数组的第 `index` 个值；
    
       
- 除此以外，`TensorArray` 还包括 `stack()` 、 `unstack()` 等常用操作

   
- 请注意，由于需要支持计算图， `tf.TensorArray` 的 `write()` 方法是不可以忽略左值的！也就是说，在图执行模式下，必须按照以下的形式写入数组：
```
arr = arr.write(index, value)
```
- 这样才可以正常生成一个计算图操作，并将该操作返回给 arr 。而不可以写成：
```
arr.write(index, value)     # 生成的计算图操作没有左值接收，从而丢失
```


In [32]:
@tf.function
def array_write_and_read():
    arr = tf.TensorArray(dtype=tf.float32, size=3)
    arr = arr.write(0, tf.constant(0.0))
    arr = arr.write(1, tf.constant(1.0))
    arr = arr.write(2, tf.constant(2.0))
    arr_0 = arr.read(0)
    arr_1 = arr.read(1)
    arr_2 = arr.read(2)
    return arr, arr_0, arr_1, arr_2


arr, a, b, c = array_write_and_read()
print(arr)
print(a, b, c)

tf.Tensor(<unprintable>, shape=(), dtype=variant)
tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(2.0, shape=(), dtype=float32)


# `tf.function` 性能

In [34]:
#### 被 tf.function 修饰的函数，除了内部机制差别，使用起来和普通函数一样
@tf.function
def add(a, b):
    return a + b

# 调用函数
add(tf.constant([1, 2]), tf.constant([3, 8]))

<tf.Tensor: id=3009, shape=(2,), dtype=int32, numpy=array([ 4, 10])>

In [35]:
# 函数内使用函数
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)


dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=3035, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

In [38]:
# 计算梯度
w = tf.Variable(np.random.randn(3, 2), dtype=tf.float32)
b = tf.Variable(np.random.randn(2), dtype=tf.float32)

x = tf.constant([[1.0, 2.0, 3.]], dtype=tf.float32)
with tf.GradientTape() as tape:
    result = dense_layer(x, w, b)
    
tape.gradient(result, [w, b])

[<tf.Tensor: id=3150, shape=(3, 2), dtype=float32, numpy=
 array([[1., 1.],
        [2., 2.],
        [3., 3.]], dtype=float32)>,
 <tf.Tensor: id=3151, shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>]

> 当图中有很多小操作时，图模式相比即时模式快很多；当图中有一些昂贵的操作，如卷积操作时，加速就不明显了

In [41]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)


@tf.function
def conv_fn(image):
    return conv_layer(image)


image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image)
conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))

Eager conv: 0.5390309999993406
Function conv: 0.5068611999995483


## 调试

即时执行模式调试代码比在 `tf.function` 内容易得多；因此将函数用 `tf.function` 装饰起来之前，应保证函数无 bug
- 调试阶段，可以使用 `tf.config.run_functions_eagerly(True)`，全局禁用计算图；然后再开启
- `print` 函数只有在计算图创建时调用，可以用来监控函数是否 `re(traced)`，即计算图是否因为输入数据类型改变等原因而重建了
- `tf.print` 用来监控执行阶段的中间值
- `tf.debugging.enable_check_numerics`用来监控是否生成了 `NaNs` 和 `Inf`


## 追踪和多态

- Python 支持动态类型，同一函数可以接受不同数据类型的参数；而TensorFlow 计算图需要静态数据类型和数据形状
- 而 `tf.function` 在接受同样的类型的数据时，会复用计算图；当输入数据的类型和形状不同时，会重新生成计算图

In [45]:
# 多态

@tf.function
def double(a):
    print("Tracing with", a)
    return a + a


print(double(tf.constant(1)))
print()
print(double(tf.constant(2))) # 相同类型数据，复用前一步计算图
print()
print(double(tf.constant(1.1))) # 不同类型数据，重建计算图
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

tf.Tensor(4, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [44]:
def f():
    print('Tracing!')
    tf.print('Executing')

# 使用函数而不是装饰器，不管输入数据，每次生成新计算图
tf.function(f)()
tf.function(f)()

Tracing!
Executing
Tracing!
Executing


In [50]:
############### .get_concrete_function 方法获取计算图

print("Obtaining concrete trace：")

# 获取特动计算图，指定数据类型
double_strings = double.get_concrete_function(
    tf.TensorSpec(shape=None, dtype=tf.string))

print("Executing traced function:")
print(double_strings(tf.constant("a")))
print()

# 与上一步相同的数据类型
print("Executing traced function agian:")
print(double_strings(a=tf.constant("b")))
print()

# 改变数据类型，报错
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Obtaining concrete trace：
Executing traced function:
tf.Tensor(b'aa', shape=(), dtype=string)

Executing traced function agian:
tf.Tensor(b'bb', shape=(), dtype=string)

Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "<ipython-input-47-1aec9a4715a6>", line 9, in assert_raises
    yield
  File "<ipython-input-50-9503431df288>", line 19, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_3350 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_3350]


In [51]:
############### input_signature 参数限制输入格式


@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32), ))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(x % 2 == 0, x // 2, 3 * x + 1)


print(next_collatz(tf.constant([1, 2])))

# 限定了一维数据，输入二维数据会报错
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-47-1aec9a4715a6>", line 9, in assert_raises
    yield
  File "<ipython-input-51-57d6c14feb7f>", line 14, in <module>
    next_collatz(tf.constant([[1, 2], [3, 4]]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))


## Python 类型参数或 Tensor 参数

- 用于控制超参和图构建的一些参数通常为 Python 类型，如 `num_layers=10,training=True,nonlinearity='relu'`；当这些参数改变时，重建图是合理的
- 不用于控制图构建的参数发生改变，重建图则非常低效了

In [52]:
def train_one_step():
    pass


@tf.function
def train(num_steps):   
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()


train(num_steps=10) # num_steps 为 Python 类型数据
train(num_steps=20) # 每次都会重建图，print 函数都会执行

Tracing with num_steps = 10
Tracing with num_steps = 20


In [54]:
train(num_steps=tf.constant(10))  # num_steps 为 Tensor 类型
train(num_steps=tf.constant(20))  # 再次调用会复用上一步创建的图

Tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


## `tf.function` 副作用

- 综上 `tf.function` 内部 Python 的函数副作用，如 print，对象改变 等只在图创建时起作用；因此只用来 调试 图的创建过程
- 此外 `tf.Variable.assign,tf.print,tf.summary` 保证每次调用时，不论是创建图还是执行图，都会正常执行  


In [57]:
@tf.function
def f(x):
    print("Traced with", x)
    tf.print("Executed with", x)


f(1)
f(1) # 调用上一步相同的图，Python 函数副作用不再起作用
f(2)

Traced with 1
Executed with 1
Executed with 1
Traced with 2
Executed with 2


- `tf.py_function` 可以确保 Python 函数副作用每次都起作用 
    - `tf.py_function` 会将输入输出都转换成 Tensor   

In [62]:
external_list = []


def side_effect(x):
    print('Python side effect')
    external_list.append(x)


@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[]) # Python 函数每次调用都会执行


f(1)
f(1) # 每次输入相同的数据，通常函数副作用会失效
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

Python side effect
Python side effect
Python side effect


In [63]:
external_list

[<tf.Tensor: id=3646, shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: id=3647, shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: id=3648, shape=(), dtype=int32, numpy=1>]

## Python 状态
- Python 许多特征，如生成器和迭代器，依靠Python内核追踪状态，在即时执行模式会正常运行
- 而在 `tf.function` 内部则出现问题

In [65]:
external_var = tf.Variable(0)


@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))  # Python 的迭代器
    tf.print("Value of external_var:", external_var)


iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)

# 不再消耗迭代器的下一个值，而是重复第一个值，图内部 next(iterator) 失效
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


## 变量

`tf.function` 会复用图，如果在其内部创建变量，复用图时，变量也会复用；因此**通常**其内部禁止创建变量    
 而即时模式每次调用则会重新创建变量   

In [66]:
@tf.function
def f(x):
    v = tf.Variable(1.0)  # 内部创建新变量，则会出错
    v.assign_add(x)
    return v


with assert_raises(ValueError):
    f(1.0)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-47-1aec9a4715a6>", line 9, in assert_raises
    yield
  File "<ipython-input-66-5990b9e714fd>", line 9, in <module>
    f(1.0)
ValueError: in converted code:

    <ipython-input-66-5990b9e714fd>:3 f  *
        v = tf.Variable(1.0)
    D:\Program\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\ops\variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    D:\Program\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\ops\variables.py:254 _variable_v2_call
        shape=shape)
    D:\Program\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\ops\variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    D:\Program\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\eager\def_function.py:413 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables 

In [67]:
# 使用外部变量，正常运行
v = tf.Variable(1.0)


@tf.function
def f(x):
    return v.assign_add(x)


print(f(1.0))  # 2.0
print(f(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [68]:
class C:
    pass


obj = C()
obj.v = None


@tf.function
def g(x):
    if obj.v is None:            # 函数第一次调用时才会执行该逻辑内语句
        obj.v = tf.Variable(1.0) # 因此保证了变量只会在在第一次调用时创建，所以不会出错
    return obj.v.assign_add(x)


print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [69]:
state = []


@tf.function
def fn(x):
    if not state:   
        state.append(tf.Variable(2.0 * x))        # 创建的变量依赖于函数传参
        state.append(tf.Variable(state[0] * 3.0)) # 同理变量只会在在第一次调用时创建，所以不会出错
    return state[0] * x * state[1]


print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0)))

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


In [70]:
state

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>]

## AutoGraph 

AutoGraph 将Python即时执行代码集合转换成图，包含控制流程 `if, for, while`

In [71]:
@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x


f(tf.random.uniform([5]))

[0.207458138 0.250728369 0.0565127134 0.900473475 0.92794323]
[0.204532236 0.245603204 0.0564526282 0.716528356 0.72963351]
[0.201727062 0.240781173 0.0563927293 0.61475426 0.62284112]
[0.199034527 0.236233428 0.0563330203 0.547465086 0.553103089]
[0.196447253 0.231934905 0.0562734976 0.498617917 0.502842307]
[0.193958566 0.227863565 0.0562141687 0.4610295 0.464349538]
[0.191562369 0.224000067 0.0561550297 0.430922925 0.433622539]
[0.189253047 0.220327288 0.0560960732 0.406092346 0.408344269]
[0.187025458 0.21682997 0.0560373031 0.385149688 0.387065887]
[0.184874892 0.213494569 0.0559787154 0.367171288 0.368828028]
[0.182797 0.210308939 0.0559203103 0.351514965 0.352966189]
[0.180787787 0.207262143 0.0558620915 0.337718397 0.339003474]
[0.178843543 0.204344422 0.0558040515 0.325438946 0.326587409]
[0.176960841 0.201546878 0.0557461902 0.314416528 0.315451056]
[0.175136492 0.198861465 0.0556885079 0.304449648 0.305387944]
[0.17336753 0.196280882 0.0556310043 0.295379341 0.296235502]


<tf.Tensor: id=4022, shape=(5,), dtype=float32, numpy=
array([0.17165123, 0.19379847, 0.05557368, 0.2870784 , 0.28786382],
      dtype=float32)>

查看 autograph 创建的代码

In [72]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f', 'f_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as f_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(x):
      ag__.converted_call(tf.print, f_scope.callopts, (x,), None, f_scope)
      x = ag__.converted_call(tf.tanh, f_scope.callopts, (x,), None, f_scope)
      return x,

    def loop_test(x):
      return ag__.converted_call(tf.reduce_sum, f_scope.callopts, (x,), None, f_scope) > 1
    x, = ag__.while_stmt(loop_test, loop_body, get_state, set_state, (x,), ('x',), ())
    do_return = True
    retval_ = f_scope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)



### 条件语句

In [74]:
@tf.function
def fizzbuzz(n):
    for i in tf.range(1, n + 1):
        print('Tracing for loop')
        if i % 15 == 0:
            print('Tracing fizzbuzz branch') # 再次调用时会被忽略
            tf.print('fizzbuzz')
        elif i % 3 == 0:
            print('Tracing fizz branch')
            tf.print('fizz')
        elif i % 5 == 0:
            print('Tracing buzz branch')
            tf.print('buzz')
        else:
            print('Tracing default branch')
            tf.print(i)

print("第一次调用：")
fizzbuzz(tf.constant(5))
print("第二次调用：")
fizzbuzz(tf.constant(20))

第一次调用：
Tracing for loop
Tracing fizzbuzz branch
Tracing fizz branch
Tracing buzz branch
Tracing default branch
1
2
fizz
4
buzz
第二次调用：
1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19
buzz


### 循环语句

In [75]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print("{}({}) contains {} nodes in its graph".format(
        f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))


@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y - x)  # Some dummy computation.
    return loss


small_data = [(1, 1)] * 3
big_data = [(1, 1)] * 10

measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

train([(1, 1), (1, 1), (1, 1)]) contains 11 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph


In [76]:
measure_graph_size(
    train,
    tf.data.Dataset.from_generator(lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(
    train,
    tf.data.Dataset.from_generator(lambda: big_data, (tf.int32, tf.int32)))

train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 5 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 5 nodes in its graph


In [77]:
batch_size = 2
seq_len = 3
feature_size = 4


def rnn_step(inp, state):
    return inp + state


@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
    # [batch, time, features] -> [time, batch, features]
    input_data = tf.transpose(input_data, [1, 0, 2])
    max_seq_len = input_data.shape[0]

    states = tf.TensorArray(tf.float32, size=max_seq_len)
    state = initial_state
    for i in tf.range(max_seq_len):
        state = rnn_step(input_data[i], state)
        states = states.write(i, state)
    return tf.transpose(states.stack(), [1, 0, 2])


dynamic_rnn(rnn_step, tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))

<tf.Tensor: id=4517, shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.23907411, 0.69274783, 0.8255179 , 0.79221475],
        [0.9259411 , 0.6930404 , 1.5047482 , 0.906649  ],
        [1.7105237 , 1.6460785 , 2.1498063 , 0.930629  ]],

       [[0.11684346, 0.71039987, 0.87097895, 0.04925954],
        [0.27159166, 1.0768443 , 1.7212101 , 0.92715   ],
        [0.8292786 , 1.919958  , 2.0388227 , 1.2462468 ]]], dtype=float32)>

# 总结：AutoGraph使用规范



1. 被`@tf.function`修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。
    - 例如使用`tf.print`而不是`print`，使用`tf.range`而不是`range`，使用`tf.constant(True)`而不是`True`.  
            
    - Python中的函数仅仅会在跟踪执行函数以创建静态图的阶段使用，普通Python函数是无法嵌入到静态计算图中的，所以 在计算图构建好之后再次调用的时候，这些Python函数并没有被计算，而TensorFlow中的函数则可以嵌入到计算图中。使用普通的Python函数会导致 被`@tf.function`修饰前【eager执行】和被@tf.function修饰后【静态图执行】的输出不一致。


2. 避免在`@tf.function`修饰的函数内部定义`tf.Variable`.
    - 如果函数内部定义了`tf.Variable`,那么在【eager执行】时，这种创建`tf.Variable`的行为在每次函数调用时候都会发生。但是在【静态图执行】时，这种创建`tf.Variable`的行为只会发生在第一步跟踪Python代码逻辑创建计算图时，这会导致被`@tf.function`修饰前【eager执行】和被`@tf.function`修饰后【静态图执行】的输出不一致。实际上，TensorFlow在这种情况下一般会报错。 
    


3. 被`@tf.function`修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。
    - 静态计算图是被编译成C++代码在TensorFlow内核中执行的。Python中的列表和字典等数据结构变量是无法嵌入到计算图中，它们仅仅能够在创建计算图时被读取，在执行计算图时是无法修改Python中的列表或字典这样的数据结构变量的。

# AutoGraph和tf.Module

- 避免在`@tf.function`修饰的函数内部定义`tf.Variable`，但是如果在函数外部定义`tf.Variable`的话，又会显得这个函数有外部变量依赖，封装不够完美。

- 基类`tf.Module`，通过继承它构建子类，可以非常方便地管理变量，还可以非常方便地管理它引用的其它Module，最重要的是，能够利用`tf.saved_model`保存模型并实现跨平台部署使用。

- `tf.keras.models.Model,tf.keras.layers.Layer` 都是继承自`tf.Module`的，提供了方便的变量管理和所引用的子模块管理的功能。

In [105]:
#### 引用外部变量的函数
x = tf.Variable(1.0, dtype=tf.float32)


#在tf.function中用input_signature限定输入张量的签名类型：shape和dtype
@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return (x)


add_print(tf.constant(3.0))
# add_print(tf.constant(3))  #输入不符合张量签名的参数将报错

4


<tf.Tensor: id=5614, shape=(), dtype=float32, numpy=4.0>

In [106]:
#### Module之类，封装变量
class DemoModule(tf.Module):
    def __init__(self, init_value=tf.constant(0.0), name=None):
        super(DemoModule, self).__init__(name=name)
        with self.name_scope:  #相当于with tf.name_scope("demo_module")
            self.x = tf.Variable(init_value, dtype=tf.float32, trainable=True)

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
    def addprint(self, a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return (self.x)

In [107]:
demo = DemoModule(init_value=tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))

6


In [108]:
result = demo.addprint(tf.constant(50.0))

56


In [109]:
#查看模块中的全部子模块
demo.submodules

()

In [111]:
#使用tf.saved_model 保存模型，并指定需要跨平台部署的方法
tf.saved_model.save(
    demo,
    "demo_model",
    signatures={"serving_default": demo.addprint},  # 
)

INFO:tensorflow:Assets written to: demo_model\assets


In [112]:
#加载模型
demo2 = tf.saved_model.load("demo_model")
demo2.addprint(tf.constant(5.0))

61


<tf.Tensor: id=5942, shape=(), dtype=float32, numpy=61.0>

In [114]:
# 查看模型文件相关信息
!saved_model_cli show --dir demo_model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['a'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: serving_default_a:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


In [115]:
import tensorflow as tf
from tensorflow.keras import models, layers, losses, metrics

In [116]:
print(issubclass(tf.keras.Model, tf.Module))
print(issubclass(tf.keras.layers.Layer, tf.Module))
print(issubclass(tf.keras.Model, tf.keras.layers.Layer))

True
True
True


In [117]:
tf.keras.backend.clear_session() 

model = models.Sequential()

model.add(layers.Dense(4,input_shape = (10,)))
model.add(layers.Dense(2))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [118]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[ 0.49823368,  0.38705254,  0.49931097, -0.47241575],
        [-0.08276552, -0.3621772 ,  0.36847925, -0.10161632],
        [-0.49198854,  0.58784914, -0.5340485 , -0.4401762 ],
        [ 0.24897873, -0.00801528, -0.08869678,  0.6431689 ],
        [-0.21808079, -0.41452947, -0.05316031,  0.33350515],
        [-0.14954305,  0.35132825, -0.04744196, -0.2250906 ],
        [ 0.4554441 , -0.18187407, -0.46764714,  0.1906746 ],
        [-0.41395682,  0.20659739,  0.41261542,  0.5152123 ],
        [-0.62937295, -0.29700363,  0.10905778,  0.10109794],
        [-0.34243053, -0.20391187,  0.4967723 , -0.31809756]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.4328966 , -0.7175617 ],
        [ 0.18615818, -0.11278558],
        [-0.14009857,  0.9860091 ],
        [ 

In [119]:
model.layers[0].trainable = False  #冻结第0层的变量,使其不可训练
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.4328966 , -0.7175617 ],
        [ 0.18615818, -0.11278558],
        [-0.14009857,  0.9860091 ],
        [ 0.09973741, -0.91382265]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[1.0078322],
        [1.2090417]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [120]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x1f0c7b3ce48>,
 <tensorflow.python.keras.layers.core.Dense at 0x1f0c79bd408>)

In [121]:
model.layers

In [122]:
print(model.name)
print(model.name_scope())

sequential
sequential
